In [61]:
import sys
import base64
import random
import chardet
import codecs
import json

import numpy as np

from re import sub
from collections import defaultdict

from collections import defaultdict, Counter

import pickle

from sklearn.feature_extraction import DictVectorizer

In [670]:
def read_x(path, dv, max_count = None):
    f = open(path, "r")
    
    p = f.readline()
    
    dict_list = []
    l_id = 0
    while p != '':
        print l_id, len(dict_list)
        dict_list  += split.get_features(p, max_count)
        
        p = f.readline()
        l_id += 1
        
    
    
    
    
    return dv.fit_transform(dict_list), dv
    

In [702]:
def read_y(path, max_count = None):
    f = open(path, "r")
    result = []
   
    
    p = f.readline()
    
    total_read = 0
    l_id = 0
    while p != '':
        p = json.loads(p)
        tmp = []
     
        for sentence in p["Sentences"]:
            n = sentence.count('.')
            if sentence[-1] == '.':
                n -= 1
            tmp += [0] * n
            if sentence[-1] == '.':
                tmp.append(1)
            total_read += n
        
            
            if max_count is not None and total_read >= max_count:
                return np.array(result)
        
#         if l_id == 77:
#             print tmp
#             for sentence in p["Sentences"]:
#                 print sentence
#                 n = sentence.count('.')
#                 print n
#                 print sentence[-1]
#                 if sentence[-1] == '.':
#                     n -= 1
# #                     tmp += [0] * n
#                 print n
                
            
#         print l_id
#         print tmp
#         print p["Paragraph"]
#         print l_id, len(result)
        result += tmp
        
        p = f.readline()
        l_id += 1
            
    
    return np.array(result)

In [703]:
import split
reload(split)

<module 'split' from 'split.pyc'>

In [704]:
# x, dv = read_x("gold.txt", DictVectorizer(sparse=False), max_count=None)

In [705]:
from sklearn.tree import DecisionTreeClassifier

In [706]:
y = read_y("gold.json", max_count = None)

In [700]:
len(y)

380

In [701]:
len(x)

380

In [656]:
y

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1,